In [1]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("../src/FEM.jl")
using .FEM

In [2]:
gmsh.open("cantilever1D.geo")

Info    : Reading 'cantilever1D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000335858s, CPU 0.000311s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Blossom: 3 internal 4 closed
Info    : Blossom recombination completed (Wall 7.1828e-05s, CPU 6.8e-05s): 2 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 1, min Q = 1
Info    : Done meshing 2D (Wall 0.00056736s, CPU 0.000526s)
Info    : 6 nodes 12 elements
Info    : Done reading 'cantilever1D.geo'


In [3]:
problem = FEM.Problem(type="PlaneStress", E=2.0e5, ν=0.0, ρ=1.0)

Main.FEM.Problem("cantilever", "PlaneStress", 2, 200000.0, 0.0, 1.0, 1.0, 6)

In [4]:
#gmsh.fltk.run()

In [5]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [6]:
K = FEM.stiffnessMatrix(problem)

12×12 SparseMatrixCSC{Float64, Int64} with 112 stored entries:
  180000.0    25000.0         ⋅          ⋅   …   -90000.0           -25000.0
   25000.0   340000.0         ⋅          ⋅       -25000.0          -170000.0
        ⋅          ⋅    180000.0   -25000.0      -90000.0            25000.0
        ⋅          ⋅    -25000.0   340000.0       25000.0          -170000.0
        ⋅          ⋅   -160000.0    25000.0       70000.0           -25000.0
        ⋅          ⋅    -25000.0  -330000.0  …    25000.0           160000.0
 -160000.0   -25000.0         ⋅          ⋅        70000.0            25000.0
   25000.0  -330000.0         ⋅          ⋅       -25000.0           160000.0
   70000.0    25000.0    70000.0   -25000.0     -320000.0                0.0
  -25000.0   160000.0    25000.0   160000.0           3.63798e-12  -660000.0
  -90000.0   -25000.0   -90000.0    25000.0  …   360000.0                0.0
  -25000.0  -170000.0    25000.0  -170000.0           0.0           680000.0

In [7]:
#M = FEM.massMatrix(problem, lumped=false)

In [8]:
#dof, dof = size(M)
#a = zeros(dof)
#a[1:2:dof] .= 10
#fx = M * a

In [9]:
supp1 = FEM.displacementConstraint("bottom", uy=0)
supp2 = FEM.displacementConstraint("left", ux=0)
load1 = FEM.load("right", fx=10)
#load2 = FEM.traction("bottom", fx=1)
#load3 = FEM.traction("top", fx=1)
load4 = FEM.load("body", fx=1)

("body", 1, 0, 0)

In [10]:
f = FEM.loadVector(problem, [load1 load4])

12-element Vector{Float64}:
 125.0
   0.0
 175.0
   0.0
 175.0
   0.0
 125.0
   0.0
 250.0
   0.0
 250.0
   0.0

In [11]:
K1, f1 = FEM.applyBoundaryConditions(problem, K, f, [supp1 supp2])

(sparse([1, 2, 3, 5, 6, 9, 11, 12, 4, 3  …  5, 6, 8, 9, 11, 3, 5, 6, 8, 12], [1, 2, 3, 3, 3, 3, 3, 3, 4, 5  …  11, 11, 11, 11, 11, 12, 12, 12, 12, 12], [1.0, 1.0, 180000.00000000003, -160000.00000000003, -25000.000000000004, 70000.00000000003, -90000.00000000001, 25000.000000000004, 1.0, -160000.00000000003  …  70000.00000000001, 25000.0, -25000.0, -320000.0, 360000.0, 25000.000000000004, -25000.000000000004, 160000.00000000006, 160000.00000000006, 680000.0], 12, 12), [0.0, 0.0, 175.0, 0.0, 175.0, 0.0, 0.0, 0.0, 250.0, 0.0, 250.0, 0.0])

In [12]:
q = FEM.solveDisplacement(K1, f1)

12-element Vector{Float64}:
  0.0
  0.0
  0.030000000000000124
  0.0
  0.030000000000000124
 -1.9871403873599e-19
  0.0
 -9.95921618758584e-20
  0.021250000000000085
  0.0
  0.021250000000000085
  2.11633343986199e-19

In [13]:
FT = K * q - f

12-element Vector{Float64}:
 -550.0000000000018
   -3.597766847765384e-14
    2.2737367544323206e-13
    7.770916924396219e-14
   -9.094947017729282e-13
   -7.982550268382418e-14
 -550.0000000000018
    3.386133503779185e-14
    1.8189894035458565e-12
   -8.89659529268771e-14
    1.8189894035458565e-12
    9.618168181271956e-14

In [14]:
S = FEM.solveStress(problem, q)

Main.FEM.StressField([[35.00000000000015; 0.0; … ; 0.0; 0.0;;], [85.00000000000034; 0.0; … ; 0.0; 0.0;;]], [7, 8], 1)

In [15]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [16]:
#u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
#uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
#uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
#s = FEM.showStressResults(problem, S, "s", name="σ", visible=true, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=false)
gmsh.view.option.setNumber(sx, "MaxRecursionLevel", 0)
#sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
#sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
#sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
#syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
#szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)
ft = FEM.showDoFResults(problem, FT, "uvec", name="F_T", visible=false)
gmsh.view.option.setNumber(ft, "MaxRecursionLevel", 0)

"ux..ok"

"sx..ok"

"uvec..ok"

In [17]:
FEM.plotOnPath(problem, "path", sx, 1000, name="σx", visible=true);
#FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false);
FEM.plotOnPath(problem, "path", ux, 1000, name="ux", visible=true);

In [18]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [19]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240121
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [20]:
gmsh.finalize()